# Run Inference across an entire area

## Install dependencies

In [ ]:
%pip install --quiet --upgrade git+https://github.com/mozilla-ai/osm-ai-helper.git

## Setup Secrets

You need to set the following secrets in the notebook.

Check the [Authorization Guide](https://mozilla-ai.github.io/osm-ai-helper/authorization) in the docs to learn how to obtain them.



In [ ]:
import os

from google.colab import userdata

os.environ["MAPBOX_TOKEN"] = userdata.get("MAPBOX_TOKEN")
os.environ["OSM_CLIENT_ID"] = userdata.get("OSM_CLIENT_ID")
os.environ["OSM_CLIENT_SECRET"] = userdata.get("OSM_CLIENT_SECRET")

## Download Trained Model

You can check the [Create Dataset](https://colab.research.google.com/github/mozilla-ai//osm-ai-helper/blob/main/demo/create_dataset.ipyn) and [Finetune Model](https://colab.research.google.com/github/mozilla-ai//osm-ai-helper/blob/main/demo/finetune_model.ipynb) notebooks to learn how to train your own model.

In [ ]:
from huggingface_hub import hf_hub_download

In [ ]:
hf_hub_download(
    "mozilla-ai/swimming-pool-detector",
    filename="model.pt",
    repo_type="model",
    local_dir="models",
)

# Run Inference

`AREA_NAME` can be city, state, country, etc.

Uses the [Nominatim API](https://nominatim.org/release-docs/develop/api/Search/).

Large areas will take more time to process.

In [ ]:
AREA_NAME = None

if AREA_NAME is None:
    raise RuntimeError("Need to provide an area name")

## Split AREA_NAME into lat_lon inputs for `run_inference`

In [ ]:
from osm_ai_helper.utils.inference import split_area_into_lat_lon_centers

In [ ]:
lat_lon_centers = split_area_into_lat_lon_centers(AREA_NAME, 18, 2)
print(f"Number of `run_inference` calls: {len(lat_lon_centers)}")
print(f"Will take aproximatelly {len(lat_lon_centers) / 10} min")

775

## Run inference across the entire area

If it takes too long to process the entire area and you want to process by parts,
you can interrupt the inference and set `ALREADY_PROCESSED` to number of the last lat_lon_center
you processed. 

In [ ]:
ALREADY_PROCESSED = 0

In [ ]:
from osm_ai_helper.run_inference import run_inference

for n, (lat, lon) in enumerate(lat_lon_centers[ALREADY_PROCESSED:]):
    print(f"{n + ALREADY_PROCESSED} of {len(lat_lon_centers)}")
    output_path, existing, new, missing = run_inference(
        "models/model.pt",
        output_dir="results",
        lat_lon=(lat, lon),
        margin=2,
        save_full_images=False,
        batch_size=64,
    )

# Aggregate results

In [ ]:
import shutil
from pathlib import Path

results = Path("results")
merged = Path("merged")
merged.mkdir(exist_ok=True, parents=True)

for location in results.iterdir():
    new_polygons = list(location.glob("*.json"))
    if not new_polygons:
        location.rmdir()
        continue
    for new_polygon in new_polygons:
        shutil.copy(new_polygon, merged / f"{location.name}-{new_polygon.name}")
        shutil.copy(
            new_polygon.with_suffix(".png"),
            merged / f"{location.name}-{new_polygon.with_suffix('.png').name}",
        )
        shutil.copy(
            f"{location}/{new_polygon.stem}_painted.png",
            merged / f"{location.name}-{new_polygon.stem}_painted.png",
        )

In [ ]:
len(list(merged.glob("*.json")))

# Manually Filter results

In [ ]:
import shutil
import time
from pathlib import Path

from matplotlib import pyplot as plt

for new_polygon in Path("merged").glob("*.json"):
    raw_image = new_polygon.with_suffix(".png")
    painted_image = f"{new_polygon.parent}/{new_polygon.stem}_painted.png"

    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    # Assuming raw_image_path and painted_image_path are image file paths
    axes[0].imshow(plt.imread(raw_image))
    axes[0].set_title("Raw Image")

    axes[1].imshow(plt.imread(painted_image))
    axes[1].set_title("Painted Image")

    plt.show()
    time.sleep(1.5)

    user_input = input("Keep image? (Y/N): ")

    if user_input.upper() == "Y":
        keep_folder = Path("keep")
        keep_folder.mkdir(parents=True, exist_ok=True)
        shutil.move(new_polygon, keep_folder / new_polygon.name)
        print(f"Images moved to {keep_folder}")
    else:
        discard_folder = Path("discard")
        discard_folder.mkdir(parents=True, exist_ok=True)
        shutil.move(new_polygon, discard_folder / new_polygon.name)
        print(f"Images moved to {discard_folder}")

# Export Results

The results will be exported in [OsmChange](https://wiki.openstreetmap.org/wiki/OsmChange) format.

You can then import the file in [any of the supported editors](https://wiki.openstreetmap.org/wiki/OsmChange#Editors) format.

In [ ]:
from osm_ai_helper.export_osm import export_osm

In [ ]:
export_osm(
    results_dir=output_path / "keep",
    output_dir="exported",
    tags={"leisure": "swimming_pool", "access": "private", "location": "outdoor"},
)